 # Classification des avis sur des vêtements de femmes vendus dans le e-commerce

- Est ce que les avis que l'on a des vêtements sont représentatifs de la note qui est attribuée ?

Idées :
- visualisation de données : quels types de vêtements ont les notes les plus élevées ?
- nb d'avis donné selon l'âge des clients

In [99]:
import pandas as pd
import spacy


## I. Import des données

Dans un premier temps, nous allons importer nos données. Notre base de données contient des informations sur des avis de vêtements femmes vendus sur internet. Ces données sont issus d'un processus de webscrapping.

In [100]:
# Import des données
data = pd.read_csv("Womens Clothing E-Commerce Reviews.csv", sep = ",")

# Renomage première colonne pour pouvoir l'utiliser comme id par la suite
data = data.rename(columns = {"Unnamed: 0" : "id"})

# Affichage des 5 premières lignes
data.head()

,id,Clothing ID,Age,Title,Review Text,Rating,Recommended IND,Positive Feedback Count,Division Name,Department Name,Class Name
0,0,767,33,NaN,Absolutely wonderful - silky and sexy and comf...,4,1,0,Initmates,Intimate,Intimates
1,1,1080,34,NaN,Love this dress! it's sooo pretty. i happene...,5,1,4,General,Dresses,Dresses
2,2,1077,60,Some major design flaws,I had such high hopes for this dress and reall...,3,0,0,General,Dresses,Dresses
3,3,1049,50,My favorite buy!,"I love, love, love this jumpsuit. it's fun, fl...",5,1,0,General Petite,Bottoms,Pants
4,4,847,47,Flattering shirt,This shirt is very flattering to all due to th...,5,1,6,General,Tops,Blouses


In [101]:
data.shape

(23486, 11)

Notre jeu de données contient 23 486 avis et 11 colonnes. 

## II. Pré-traitement des données

Pour avoir des données plus propres, nous allons effectuer divers pré-traitements.

Pour commencer, nous allons supprimer les lignes où nous avons des valeurs manquantes. 

In [102]:
# Suppression des valeurs manquantes
data = data.dropna()
data.shape

(19662, 11)

Suite à cette manipulation, nous supprimons environ 4 000 lignes pour pouvoir avoir des données complètes pour poursuivre notre analyse.

### Récupération des données pour notre étude

### Traitement de la casse

- suppression des valeurs manquantes => pas d'avis : inutile
- suppression des caractères spéciaux 
- suppression des majuscules
- suppression des mots vides
- lemmatisation 
- affiche du nombre de mots par étiquette grammaticale
- extraction des mots (groupes de mots) les plus fréquents

- wordcloud

Dans un premier temps, nous allons récupérer les avis.

In [103]:
avis = data["Review Text"]
avis

2        I had such high hopes for this dress and reall...
3        I love, love, love this jumpsuit. it's fun, fl...
4        This shirt is very flattering to all due to th...
5        I love tracy reese dresses, but this one is no...
6        I aded this in my basket at hte last mintue to...
                               ...                        
23481    I was very happy to snag this dress at such a ...
23482    It reminds me of maternity clothes. soft, stre...
23483    This fit well, but the top was very see throug...
23484    I bought this dress for a wedding i have this ...
23485    This dress in a lovely platinum is feminine an...
Name: Review Text, Length: 19662, dtype: object

In [104]:
type(avis)

pandas.core.series.Series

Nous allons récupérer seulement la partie textuelle de l'avis, cela nous permet de ne avoir un objet Pandas.Series.

In [105]:
from pprint import pprint

liste_avis = data["Review Text"].values.tolist()
pprint(liste_avis[:10])

['I had such high hopes for this dress and really wanted it to work for me. i '
 'initially ordered the petite small (my usual size) but i found this to be '
 'outrageously small. so small in fact that i could not zip it up! i reordered '
 'it in petite medium, which was just ok. overall, the top half was '
 'comfortable and fit nicely, but the bottom half had a very tight under layer '
 'and several somewhat cheap (net) over layers. imo, a major design flaw was '
 'the net over layer sewn directly into the zipper - it c',
 "I love, love, love this jumpsuit. it's fun, flirty, and fabulous! every time "
 'i wear it, i get nothing but great compliments!',
 'This shirt is very flattering to all due to the adjustable front tie. it is '
 'the perfect length to wear with leggings and it is sleeveless so it pairs '
 'well with any cardigan. love this shirt!!!',
 'I love tracy reese dresses, but this one is not for the very petite. i am '
 'just under 5 feet tall and usually wear a 0p in this 

Grâce à cette manipulation, chaque avis est élément d'une liste d'avis. 

Ensuite, nous allons découper les avis en liste de mots et les mettre en minuscules pour pouvoir les analyser plus facilement.

In [106]:
# Découpage des avis en mots

liste_avis_clean = []

for avis in liste_avis : 
    avis = str(avis)
    avis_clean = avis.split()
    avis_clean = avis.lower()
    liste_avis_clean.append(avis_clean)

liste_avis_clean[:10]

['i had such high hopes for this dress and really wanted it to work for me. i initially ordered the petite small (my usual size) but i found this to be outrageously small. so small in fact that i could not zip it up! i reordered it in petite medium, which was just ok. overall, the top half was comfortable and fit nicely, but the bottom half had a very tight under layer and several somewhat cheap (net) over layers. imo, a major design flaw was the net over layer sewn directly into the zipper - it c',
 "i love, love, love this jumpsuit. it's fun, flirty, and fabulous! every time i wear it, i get nothing but great compliments!",
 'this shirt is very flattering to all due to the adjustable front tie. it is the perfect length to wear with leggings and it is sleeveless so it pairs well with any cardigan. love this shirt!!!',
 'i love tracy reese dresses, but this one is not for the very petite. i am just under 5 feet tall and usually wear a 0p in this brand. this dress was very pretty out of

Puis, nous allons tockeniser notre texte. 

In [107]:
from nltk.tokenize import RegexpTokenizer

tokenizer = RegexpTokenizer('\w+')
liste_avis_clean = [tokenizer.tokenize(str(avis)) for avis in liste_avis_clean]
liste_avis_clean[:10]

[['i',
  'had',
  'such',
  'high',
  'hopes',
  'for',
  'this',
  'dress',
  'and',
  'really',
  'wanted',
  'it',
  'to',
  'work',
  'for',
  'me',
  'i',
  'initially',
  'ordered',
  'the',
  'petite',
  'small',
  'my',
  'usual',
  'size',
  'but',
  'i',
  'found',
  'this',
  'to',
  'be',
  'outrageously',
  'small',
  'so',
  'small',
  'in',
  'fact',
  'that',
  'i',
  'could',
  'not',
  'zip',
  'it',
  'up',
  'i',
  'reordered',
  'it',
  'in',
  'petite',
  'medium',
  'which',
  'was',
  'just',
  'ok',
  'overall',
  'the',
  'top',
  'half',
  'was',
  'comfortable',
  'and',
  'fit',
  'nicely',
  'but',
  'the',
  'bottom',
  'half',
  'had',
  'a',
  'very',
  'tight',
  'under',
  'layer',
  'and',
  'several',
  'somewhat',
  'cheap',
  'net',
  'over',
  'layers',
  'imo',
  'a',
  'major',
  'design',
  'flaw',
  'was',
  'the',
  'net',
  'over',
  'layer',
  'sewn',
  'directly',
  'into',
  'the',
  'zipper',
  'it',
  'c'],
 ['i',
  'love',
  'love',
 

Ensuite, on supprime la ponctuation.

In [108]:
import string
punct = string.punctuation

# Pour chaque token dans chaque avis, si le token n'est pas dans la liste des ponctuations, on le garde
liste_avis_clean = [[token for token in avis if token not in punct] for avis in liste_avis_clean]
liste_avis_clean[:10]

[['i',
  'had',
  'such',
  'high',
  'hopes',
  'for',
  'this',
  'dress',
  'and',
  'really',
  'wanted',
  'it',
  'to',
  'work',
  'for',
  'me',
  'i',
  'initially',
  'ordered',
  'the',
  'petite',
  'small',
  'my',
  'usual',
  'size',
  'but',
  'i',
  'found',
  'this',
  'to',
  'be',
  'outrageously',
  'small',
  'so',
  'small',
  'in',
  'fact',
  'that',
  'i',
  'could',
  'not',
  'zip',
  'it',
  'up',
  'i',
  'reordered',
  'it',
  'in',
  'petite',
  'medium',
  'which',
  'was',
  'just',
  'ok',
  'overall',
  'the',
  'top',
  'half',
  'was',
  'comfortable',
  'and',
  'fit',
  'nicely',
  'but',
  'the',
  'bottom',
  'half',
  'had',
  'a',
  'very',
  'tight',
  'under',
  'layer',
  'and',
  'several',
  'somewhat',
  'cheap',
  'net',
  'over',
  'layers',
  'imo',
  'a',
  'major',
  'design',
  'flaw',
  'was',
  'the',
  'net',
  'over',
  'layer',
  'sewn',
  'directly',
  'into',
  'the',
  'zipper',
  'it',
  'c'],
 ['i',
  'love',
  'love',
 

## III. Classification

### Traitement et séparation des données

Dans cette partie, nous allons chercher à classifier les avis en fonction de leur note. 

Nous allons utiliser la colonne "Rating" comme étiquettes et "Review Text" comme valeurs. 




#### Sélection des informations dans notre dataframe

Nous allons sélectionner les trois colonnes qui vont nous servir pour la classification dans un objectif d'optimiser les temps de calculs et de ne pas avoir d'informations superflus.

In [109]:
# On récupère la colonne id, Rating et Review Text
data = data[["id", "Rating", "Review Text"]]
data.head()

,id,Rating,Review Text
2,2,3,I had such high hopes for this dress and reall...
3,3,5,"I love, love, love this jumpsuit. it's fun, fl..."
4,4,5,This shirt is very flattering to all due to th...
5,5,2,"I love tracy reese dresses, but this one is no..."
6,6,5,I aded this in my basket at hte last mintue to...


#### Analyse de la colonne "Rating"

In [110]:
# Analyse de la colonne "Rating"
data["Rating"].value_counts()

Rating
5    10858
4     4289
3     2464
2     1360
1      691
Name: count, dtype: int64

Nous avons ici des notes allant de 1 à 5. Nous allons diviser ces valeurs en 3 catégories : 
- -1 pour les notes allant de 1 à 2
- 0 pour les notes égales à 3 
- 1 pour les plus élevées (4 et 5)

#### Analyse de la colonne "Review Text"

Notre colonne correspondant aux valeurs est "Review Text". \
Cette colonne contient tous les avis laissés par les internautes sur les différents vêtements.

#### Changement des étiquettes

Pour réaliser notre classification, nous allons donc modifier les étiquettes comme précisé ci-dessus. 

In [111]:
def map_label_to_numeric(label):
    return 1 if label == 5 else 0 if label == 3 or label == 4 else -1

In [112]:
def get_labels(data):
    labels = data[["id","Rating"]]
    labels['Rating'] = labels['Rating'].apply(map_label_to_numeric)
    labels.set_index('id', inplace=True)
    
    # ajouter les labels dans data selon l'id
    data['score_avis'] = labels

    # data['score_avis'] = labels
    return data

In [113]:
data = get_labels(data)
data.head()

C:\Users\ASUS\AppData\Local\Temp\ipykernel_7908\29256686.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  labels['Rating'] = labels['Rating'].apply(map_label_to_numeric)


,id,Rating,Review Text,score_avis
2,2,3,I had such high hopes for this dress and reall...,0
3,3,5,"I love, love, love this jumpsuit. it's fun, fl...",1
4,4,5,This shirt is very flattering to all due to th...,1
5,5,2,"I love tracy reese dresses, but this one is no...",-1
6,6,5,I aded this in my basket at hte last mintue to...,1


In [114]:
# On analyse la nouvelle colonne "score_avis"
data["score_avis"].value_counts()

score_avis
 1    10858
 0     6753
-1     2051
Name: count, dtype: int64

Grâce à cette manipulation, nous pouvons observer que les avis ayant la note de 5 sont majoritaires dans notre jeu de données puisque cela correspond à la note de 1. Les avis compris entre 1 et 2 ont une proportion plus faible (-1). 

A présent, nous n'avons plus besoin de la colonne Rating, nous pouvons donc la supprimer du dataframe.

In [115]:
data = data[["id", "Review Text", "score_avis"]]
data.head()

,id,Review Text,score_avis
2,2,I had such high hopes for this dress and reall...,0
3,3,"I love, love, love this jumpsuit. it's fun, fl...",1
4,4,This shirt is very flattering to all due to th...,1
5,5,"I love tracy reese dresses, but this one is no...",-1
6,6,I aded this in my basket at hte last mintue to...,1


#### Division de notre dataframe

Pour réaliser notre classification, nous avons besoin de séparer notre jeu de données en un jeu d'apprentissage, de validation et de test.

Note :
Les données en apprentissage automatique sont généralement séparées en trois jeux :
+ **entraînement** : données destinées à l'apprentissage du modèle ;
+ **validation** : données destinées à une évaluation intermédiaire du modèle pour permettre l'ajustement de ses hyperparamètres. Une fois les hyperparamètres du modèle arrêtés, on peut le ré-entraîner sur l'ensemble des données (entraînement + validation) avant de le tester sur le jeu de test ;
+ **test** : données destinées EXCLUSIVEMENT à l'évaluation FINALE (à réaliser une fois uniquement !) du modèle choisi finalement. Elles ne doivent sous aucune forme servir à la conception du modèle. Il est donc interdit aussi bien de les examiner que d'évaluer le modèle en cours de développement sur ce jeu de données.

Pour créer l'ensemble de validation, nous allons effectuer la manipulation à la fin du pré-traitement réalisé lors de la classification. 

In [ ]:
# Fonction pour diviser de notre jeu de données en 2 : train et test
def split_data(data, train_ratio):
    data_train = data.sample(frac = train_ratio)
    data_test = data.drop(data_train.index)
    return data_train, data_test

# Diviser notre jeu de données en 2 : train et test
data_train, data_test = split_data(data, 0.6)

In [117]:
data_train.shape, data_test.shape

((11797, 3), (5243, 3), (2622, 3))

Dans notre cas :
+ entraînement (appelé *Train*) contenant 11797 observations ;
+ validation (appelé *Validation*) contenant 5243 observations ;
+ test (appelé *Test*), contenant 2622 observations, soit environ 22% de la taille du jeu d'entraînement.

In [118]:
data_train.head()

,id,Review Text,score_avis
5179,5179,I have loved ag stevie ankle denim for awhile ...,1
12420,12420,"So this top is kind of short, but that is pict...",1
20169,20169,I tried this on in-store in white and in grey....,1
20161,20161,"Lovely color and sweet top, but the fabric was...",0
5969,5969,"Definitely size down, the waist and chest were...",0


In [120]:
data_test.head()

,id,Review Text,score_avis
8,8,I love this dress. i usually get an xs but it ...,1
24,24,"I love this shirt because when i first saw it,...",1
25,25,"Loved the material, but i didnt really look at...",0
42,42,This poncho is so cute i love the plaid check ...,1
48,48,"This sweater is perfect for fall...it's roomy,...",1


### Exploration des données

#### Distribution des classes

Il est important de connaître la répartition des classes dans les données d'entraînement pour pouvoir procéder à notre classification.

In [121]:
# Analyse de la colonne "score_avis" de notre jeu de données d'entrainement
print(data_train["score_avis"].value_counts())

# Calcul des proportions de chaque classe dans notre jeu de données d'entrainement
data_train["score_avis"].value_counts()/len(data_train)

score_avis
 1    6492
 0    4076
-1    1229
Name: count, dtype: int64


score_avis
 1    0.550309
 0    0.345512
-1    0.104179
Name: count, dtype: float64

Nous pouvons observer que les classes de scores sont réparties de manière aléatoire dans notre jeu d'apprentissage. Nous pouvons noter plus de 50% d'avis très favorables, correspondant à la note de 5/5. Les avis négatifs sont en minorité dans notre jeu d'entraînement.

#### Exploration du texte

Pour se faire une idée des textes auxquels nous avons affaire, nous allons les afficher pour savoir quels pré-traitements sont nécessaires.

In [122]:
# Affichage des 5 premiers avis
data_train["Review Text"].values[:5]

array(['I have loved ag stevie ankle denim for awhile now and have never been disappointed with the fit or quality. this pair is no exception to that. i am 5\'5" and about 113, slender with athletic thighs and the 26 fit great. buying them on-line, i wasn\'t able to check how distressed the denim was. the pair i received in the mail was indeed slightly different in the distressed locations from the on-line image. mine had no distressing on the upper thigh and one medium and one small spot on each leg. onl',
       "So this top is kind of short, but that is pictured. it hits right where it does in the photo, which i like. it is a sheer material, not terribly so, but you will most likely need a cami, which you could pair a long one with to offset the shorter top if you don't like the shortness. i love the top, it's very pretty detailing, cute, just as pictured. i ordered a size 2 and it fits great. i usually wear petites, but you really don't need to get a petite because it's short as it

## Représentation des textes

### Sélection de descripteurs : prétraitements textuels

#### Exemple sur un avis

In [123]:
tw = data_train['Review Text'].iloc[100]
tw

'So, i\'m really surprised that reviewers said the fit was too small and too tight in the arms. i\'m 5\'7" and 183lbs and my husband (god bless his soul) bought me an 8 for my birthday. i tried it on just to see and was pretty impressed i could button it across my chest and get it over my arms as those are common problem areas for me. regardless i need to exchange the shirt for a larger size because even though i got it on its not how is supposed to look. the shirt is loose around the midsection whi'

Pour effectuer nos tâches de traitements de langage, nous allons transformer le 1er avis en utilisant spacy. En effet, spacy a plusieurs fonctionnalités : 
- permet de tokeniser directement notre texte
- peut lemmatiser les mots
- identifier et classer les entités nommées
- analyser les dépendances syntaxiques entre les mots
- représenter les mots sous forme de vecteurs (embedding)
- etc

In [124]:
# Pour la langue anglaise
!pip install -U spacy
! python -m spacy validate

import spacy
!python -m spacy download en_core_web_sm

nlp = spacy.load('en_core_web_sm')


⠙ Loading compatibility table...
⠹ Loading compatibility table...
⠸ Loading compatibility table...
⠼ Loading compatibility table...
✔ Loaded compatibility table

================= Installed pipeline packages (spaCy v3.7.2) =================
ℹ spaCy installation:
c:\Users\ASUS\anaconda3\Lib\site-packages\spacy

NAME              SPACY            VERSION                            
en_core_web_sm    >=3.7.2,<3.8.0   3.7.1   ✔
fr_core_news_sm   >=3.7.0,<3.8.0   3.7.0   ✔

     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
      --------------------------------------- 0.2/12.8 MB 5.3 MB/s eta 0:00:03
     - -------------------------------------- 0.3/12.8 MB 3.4 MB/s eta 0:00:04
     - -------------------------------------- 0.5/12.8 MB 3.7 MB/s eta 0:00:04
     - -------------------------------------- 0.5/12.8 MB 3.2 MB/s eta 0:00:04
     -- ------------------------------------- 0.7/12.8 MB 3.4 MB/s eta 0:00:04
     ---- ----------------------------------- 1.3/12.8 MB 

In [125]:
# Transformation du premier avis en objet spacy
avis_nlp = nlp(avis) 
avis_nlp

This dress in a lovely platinum is feminine and fits perfectly, easy to wear and comfy, too! highly recommend!

In [126]:
type(avis_nlp)

spacy.tokens.doc.Doc

On a bien un objet de type spacy.

On affiche chaque token de notre objet spacy :

In [127]:
for token in avis_nlp:
    print(token)

This
dress
in
a
lovely
platinum
is
feminine
and
fits
perfectly
,
easy
to
wear
and
comfy
,
too
!
highly
recommend
!


Ensuite, nous allons pouvoir afficher les lemmes de chaque token. Grâce à cette étape, nous allons pouvoir simplifier les mots pour faciliter notre analyse textuelle.

In [128]:
# Lemmatisation
for token in avis_nlp:
    print(token.lemma_)

this
dress
in
a
lovely
platinum
be
feminine
and
fit
perfectly
,
easy
to
wear
and
comfy
,
too
!
highly
recommend
!


#### Généralisation de la lemmatisation


Pour lemmatiser notre texte, nous allons définir une fonction. Cette étape est indispensable pour récupérer les lemmes des mots de notre texte d'origine. Nous allons simplifier notre texte grâce à cette fonction.

Cette fonction va nous permettre de généraliser par la suite nos manipulations.

In [129]:
def lemmatise_text(text):
    text = nlp(text) # on transforme le texte en objet spacy
    lemmas = [token.lemma_ for token in text] # on récupère les lemmes
    return ' '.join(lemmas) # on retourne les lemmes sous forme de texte

In [130]:
lemmatise_text(avis) # On applique la fonction à notre avis

'this dress in a lovely platinum be feminine and fit perfectly , easy to wear and comfy , too ! highly recommend !'

Nous allons ensuite ajouter une colonne avec la fonction de **lemmatisation** appliquée à nos avis.

In [131]:
data_train['lemmas'] = data_train['Review Text'].apply(lemmatise_text)

In [132]:
data_train.head()

,id,Review Text,score_avis,lemmas
5179,5179,I have loved ag stevie ankle denim for awhile ...,1,I have love ag stevie ankle denim for awhile n...
12420,12420,"So this top is kind of short, but that is pict...",1,"so this top be kind of short , but that be pic..."
20169,20169,I tried this on in-store in white and in grey....,1,I try this on in - store in white and in grey ...
20161,20161,"Lovely color and sweet top, but the fabric was...",0,"lovely color and sweet top , but the fabric be..."
5969,5969,"Definitely size down, the waist and chest were...",0,"definitely size down , the waist and chest be ..."


On effectue la même manipulation sur l'ensemble de test .

In [133]:
data_test['lemmas'] = data_test['Review Text'].apply(lemmatise_text)

KeyboardInterrupt: 

In [ ]:
data_test.shape

(2622, 4)

In [ ]:
# Sauvegarde
data_train.to_pickle('train.pkl')
data_test.to_pickle('test.pkl')

##### Racines

Pour réduire les mots à leur forme de base, nous allons utiliser SnowballStemmer sur notre texte. Pour cela, nous allons créer une fonction et l'appliquer à nos avis. 

In [ ]:
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer

In [ ]:
stemmer = SnowballStemmer('english')
stemmer.tokenize(avis)

['I',
 'cannot',
 'say',
 'enough',
 'about',
 'these',
 'pajama',
 'pants',
 '.',
 "they're",
 'beautiful',
 'and',
 'crazy',
 'comfortable',
 '.',
 "it's",
 'a',
 'nice',
 'change',
 'from',
 'black',
 'or',
 'grey',
 '.',
 'i',
 'also',
 'love',
 'that',
 'there',
 'are',
 'no',
 'pockets',
 'because',
 'i',
 'hate',
 'how',
 'they',
 'jut',
 'out',
 'on',
 'me',
 '.',
 'i',
 'wanted',
 'a',
 'petite',
 'l',
 'because',
 'i',
 'am',
 'short',
 ',',
 'but',
 'the',
 'regular',
 'large',
 'is',
 'fine',
 '.',
 'i',
 'just',
 'wear',
 'them',
 'higher',
 'up',
 'on',
 'my',
 'hips',
 '.',
 'i',
 'normally',
 'wait',
 'for',
 'sales',
 'on',
 'sleepwear',
 ',',
 'but',
 'i',
 "couldn't",
 'resist',
 'on',
 'these',
 '.',
 "they're",
 'well',
 'worth',
 'the',
 'investment',
 '!']

In [ ]:
def stem_text(text):
    stemmer = SnowballStemmer('english')
    stems = [stemmer.stem(token) for token in stemmer.tokenize(text)]
    return ' '.join(stems)

In [ ]:
stem_text(avis)

'this dress in a love platinum is feminin and fit perfect , easi to wear and comfi , too ! high recommend !'

On applique la fonction à notre dataframe.

In [ ]:
data_train['stems'] = data_train['Review Text'].apply(stem_text)

In [ ]:
data_train.head()

,id,Review Text,score_avis,lemmas,stems
13301,13301,This top is so pretty and easy to wear. the ma...,1,this top be so pretty and easy to wear . the m...,this top is so pretti and easi to wear . the m...
6121,6121,I've been looking for a new winter dress and t...,1,I have be look for a new winter dress and this...,i'v been look for a new winter dress and this ...
14393,14393,The material is so soft and i really like the ...,0,the material be so soft and I really like the ...,the materi is so soft and i realli like the de...
6804,6804,This hoodie has a great fit! it is slightly ta...,1,this hoodie have a great fit ! it be slightly ...,this hoodi has a great fit ! it is slight tape...
2048,2048,I'm 5'6 and between 100-105lbs. buying clothes...,1,I be 5'6 and between 100 - 105lbs . buy clothe...,i'm 5 ' 6 and between 100-105 lbs . buy cloth ...


On fait de même sur l'ensemble de test.

In [ ]:
data_test['stems'] = data_test['Review Text'].apply(stem_text)

In [ ]:
data_test.shape

(2622, 5)

In [ ]:
# Sauvegarde
data_train.to_pickle('train.pkl')
data_test.to_pickle('test.pkl')

##### Étiquettes morphosyntaxiques

Puis, nous allons analyser notre texte et renvoyer chaque mot remplacé par sa catégorie grammaticale pour continuer l'étude des avis.

In [ ]:
def replace_words_with_pos_tag(text):
    text = nlp(text)
    return ' '.join([token.pos_ for token in text])

On teste la fonction sur un avis.

In [ ]:
replace_words_with_pos_tag(avis)

'DET NOUN ADP DET ADJ NOUN AUX ADJ CCONJ VERB ADV PUNCT ADJ PART VERB CCONJ ADJ PUNCT ADV PUNCT ADV VERB PUNCT'

On applique la fonction à notre ensemble d'entrainement et on ajoute une colonne à notre dataframe.

In [ ]:
data_train['pos'] = data_train['Review Text'].apply(replace_words_with_pos_tag)

In [ ]:
data_train.head()

,id,Review Text,score_avis,lemmas,stems,pos
13301,13301,This top is so pretty and easy to wear. the ma...,1,this top be so pretty and easy to wear . the m...,this top is so pretti and easi to wear . the m...,DET NOUN AUX ADV ADJ CCONJ ADJ PART VERB PUNCT...
6121,6121,I've been looking for a new winter dress and t...,1,I have be look for a new winter dress and this...,i'v been look for a new winter dress and this ...,PRON AUX AUX VERB ADP DET ADJ NOUN NOUN CCONJ ...
14393,14393,The material is so soft and i really like the ...,0,the material be so soft and I really like the ...,the materi is so soft and i realli like the de...,DET NOUN AUX ADV ADJ CCONJ PRON ADV VERB DET N...
6804,6804,This hoodie has a great fit! it is slightly ta...,1,this hoodie have a great fit ! it be slightly ...,this hoodi has a great fit ! it is slight tape...,DET NOUN VERB DET ADJ NOUN PUNCT PRON AUX ADV ...
2048,2048,I'm 5'6 and between 100-105lbs. buying clothes...,1,I be 5'6 and between 100 - 105lbs . buy clothe...,i'm 5 ' 6 and between 100-105 lbs . buy cloth ...,PRON AUX NUM CCONJ ADP NUM PUNCT NUM PUNCT VER...


On effectue la même manipulation sur notre ensemble de test.

In [ ]:
data_test['pos'] = data_test['Review Text'].apply(replace_words_with_pos_tag)

In [ ]:
data_test.shape

(2622, 6)

In [ ]:
# Sauvegarde
data_train.to_pickle('train.pkl')
data_test.to_pickle('test.pkl')

#####  Classe d'appartenance des entités nommées

Pour pouvoir effectuer la reconnaissances d'entités nommées sur nos avis, nous allons retourner une version de notre avis ou chaque entité nommée est remplacée par son type d'entité.

In [136]:
def ner(text):

    text = nlp(text) # on transforme le texte en objet spacy
    
    new_text = [] # on crée une liste vide

    for token in text: # pour chaque token dans l'avis
        if token.ent_iob_ == "O": # si l'entité ne fait pas partie d'une entité nommée
            new_text.append(token.text) # on ajoute le texte du token à la liste
        elif token.ent_iob_ == "B": # si l'entité fait partie d'une entité nommée
            new_text.append(token.ent_type_) # on ajoute le type de l'entité à la liste

        # Si l'entité comprend plusieurs mot on ne répète pas l'étiquette
        else:
            continue
    return ' '.join(new_text) # on retourne les étiquettes sous forme de texte

On applique la fonction sur un avis.

In [137]:
print(avis)
print("Avec les entités nommées : ", ner(avis))

This dress in a lovely platinum is feminine and fits perfectly, easy to wear and comfy, too! highly recommend!
Avec les entités nommées :  This dress in a lovely platinum is feminine and fits perfectly , easy to wear and comfy , too ! highly recommend !


On applique ensuite notre fonction à notre dataframe d'entrainement. 

In [138]:
data_train['entites_nommees'] = data_train['Review Text'].apply(ner)

In [ ]:
data_train.head()

,id,Review Text,score_avis,lemmas,stems,pos,entites_nommees
13301,13301,This top is so pretty and easy to wear. the ma...,1,this top be so pretty and easy to wear . the m...,this top is so pretti and easi to wear . the m...,DET NOUN AUX ADV ADJ CCONJ ADJ PART VERB PUNCT...,This top is so pretty and easy to wear . the m...
6121,6121,I've been looking for a new winter dress and t...,1,I have be look for a new winter dress and this...,i'v been look for a new winter dress and this ...,PRON AUX AUX VERB ADP DET ADJ NOUN NOUN CCONJ ...,I 've been looking for a new DATE dress and th...
14393,14393,The material is so soft and i really like the ...,0,the material be so soft and I really like the ...,the materi is so soft and i realli like the de...,DET NOUN AUX ADV ADJ CCONJ PRON ADV VERB DET N...,The material is so soft and i really like the ...
6804,6804,This hoodie has a great fit! it is slightly ta...,1,this hoodie have a great fit ! it be slightly ...,this hoodi has a great fit ! it is slight tape...,DET NOUN VERB DET ADJ NOUN PUNCT PRON AUX ADV ...,This ORG has a great fit ! it is slightly tape...
2048,2048,I'm 5'6 and between 100-105lbs. buying clothes...,1,I be 5'6 and between 100 - 105lbs . buy clothe...,i'm 5 ' 6 and between 100-105 lbs . buy cloth ...,PRON AUX NUM CCONJ ADP NUM PUNCT NUM PUNCT VER...,I 'm CARDINAL and CARDINAL . buying clothes fr...


Aussi, on applique à notre ensemble de test.

In [ ]:
data_test['entites_nommees'] = data_test['Review Text'].apply(ner)

In [ ]:
data_test.shape

(2622, 7)

In [ ]:
data_train.to_pickle('train.pkl')
data_test.to_pickle('test.pkl')

###  Calcul des valeurs des descripteurs

Pour procéder aux calculs, nous allons séparer notre jeu de données d'entraînement pour avoir un jeu de données de validation. Les données test nou servirons pour l'évaluation finale des modèles. 

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(data_train['Review Text'],
                                                      data_train['score_avis'],
                                                      train_size=0.75,
                                                      random_state=5)

In [ ]:
X_train.shape, X_valid.shape

((8847,), (2950,))

On a donc 8 847 lignes dans notre jeu d'entrainement et 2 950 dans celui de validation.

In [ ]:
y_train

20304    1
13051    0
10628   -1
4648     0
19493    0
        ..
19357    1
9412     0
20438    0
22105    1
11296    1
Name: score_avis, Length: 8847, dtype: int64

Nous pouvons observer que les sorties à prédire correspondent aux trois étiquettes que nous avons défini plus haut.

Pour évaluer notre modèle, nous initialisons les ensembles de test.

In [139]:
# On récupère les avis et les labels du jeu de données de test
X_test, y_test = data_test['Review Text'], data_test['score_avis'] 

### Binaire : présence/absence

In [140]:
from sklearn.feature_extraction.text import CountVectorizer

bin_count = CountVectorizer(binary=True)

In [ ]:
bin_count.fit(X_train)
bin_count

CountVectorizer(binary=True)

In [141]:
X_train_vectorized_bin = bin_count.transform(X_train)
X_train_vectorized_bin

NameError: name 'X_train' is not defined

In [ ]:
X_valid_vectorized_bin = bin_count.transform(X_valid)
X_test_vectorized_bin = bin_count.transform(X_test)

In [ ]:
X_valid_vectorized_bin # MEME NOMBRE DE COLONNES QUE X_train_vectorized_bin

<2950x9677 sparse matrix of type '<class 'numpy.int64'>'
	with 128245 stored elements in Compressed Sparse Row format>

###  Numérique discret : décomptes d'occurrence

Nous allons calculer les fréquences d'occurence des termes dans nos avis. 

In [ ]:
vect_count = CountVectorizer().fit(X_train)

Nous pouvons examiner le vocabulaire de nos avis : 

In [ ]:
vect_count.get_feature_names_out()[:50] # 50 premiers mots ("types" du vocabulaire)

array(['00', '00p', '03', '03dd', '0in', '0p', '0petite', '0r', '0verall',
       '0xs', '10', '100', '1000', '100lb', '100lbs', '102', '102lbs',
       '103', '103lb', '103lbs', '104', '105', '105lbs', '106', '107',
       '107lb', '107pound', '108', '108lbs', '109', '109lbs', '10lbs',
       '10p', '10x', '11', '110', '110lbs', '111', '111lbs', '112',
       '112lbs', '113', '114', '114lb', '115', '115lbs', '116', '116lb',
       '116lbs', '117'], dtype=object)

In [ ]:
vect_count.get_feature_names_out()[-50:] # 50 derniers mots ("types" du vocabulaire)

array(['yep', 'yes', 'yest', 'yesterday', 'yet', 'yey', 'yfit', 'yiddish',
       'yield', 'yikes', 'yippee', 'yo', 'yoga', 'yogini', 'yogis',
       'yoke', 'yolk', 'york', 'you', 'young', 'younger', 'your', 'youre',
       'yours', 'yourself', 'yourselves', 'youthful', 'youthfull', 'yr',
       'yrs', 'yuck', 'yucky', 'yummiest', 'yummy', 'zag', 'zermatt',
       'zero', 'zig', 'zigzag', 'zip', 'zipped', 'zipper', 'zippered',
       'zippers', 'zipping', 'zips', 'zombie', 'zone', 'zoom', 'zuma'],
      dtype=object)

Taille de notre vocabulaire :

In [142]:
len(vect_count.get_feature_names_out()) 

NameError: name 'vect_count' is not defined

#### Création matrice document-termes

Nous allons créer la matrice document-termes avec le même vectoriseur.

In [ ]:
X_train_vectorized_count = vect_count.transform(X_train)
X_train_vectorized_count

<8847x9677 sparse matrix of type '<class 'numpy.int64'>'
	with 388140 stored elements in Compressed Sparse Row format>

In [ ]:
X_valid_vectorized_count = vect_count.transform(X_valid)
X_test_vectorized_count = vect_count.transform(X_test)

A présent, nous allons prendre en compte les bi-grammes dans notre vocabulaire. 

In [ ]:
vect_count_bigrams = CountVectorizer(min_df=5, ngram_range=(1,2)).fit(X_train)
X_train_vectorized_count_bigrams = vect_count_bigrams.transform(X_train)
X_valid_vectorized_count_bigrams = vect_count_bigrams.transform(X_valid)
X_test_vectorized_count_bigrams = vect_count_bigrams.transform(X_test)

In [ ]:
len(vect_count_bigrams.get_feature_names_out())

17411

Nous avons presque 2 fois plus de vocabulaire avec inclusion des bigrammes.

###  Numérique continu : TF-IDF (ou autres pondérations)

Nous allons limiter le vocabulaire à des termes qui apparaissent au moins 5 fois dans le document.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
vect_tfidf = TfidfVectorizer(min_df=5).fit(X_train)

In [ ]:
len(vect_count.get_feature_names_out()), len(vect_tfidf.get_feature_names_out())

(9677, 3260)

La réduction de la taille du vocadulaire est importante et est due au paramètre min_df=5 : on a quasiment 3 fois moins de termes !

Nous allons vectoriser les jeux de données. 

In [ ]:
# Vectorisation des corpus d'entrainement, de validation et de test
X_train_vectorized_tfidf = vect_tfidf.transform(X_train)
X_valid_vectorized_tfidf = vect_tfidf.transform(X_valid)
X_test_vectorized_tfidf = vect_tfidf.transform(X_test)

## Classification des textes

Nous allons réaliser une classification en utilisant plusieurs modèles afin de comparer les performances. 